In [1]:
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def KMer(text, k):
    kmers_count = {}
    s = 0
    for i in range(len(text) - k + 1):
        kmer = text[i:i + k]
        s += 1
        if kmer in kmers_count:
            kmers_count[kmer] += 1
        else:
            kmers_count[kmer] = 1
    for key, value in kmers_count.items():
        kmers_count[key] = value / s

    return kmers_count

def codon(text, k):
    codon_count = {}
    s = 0
    i = 0
    while i < len(text) - (k * 3) + 1:
        kmer = text[i:i + (k * 3)]
        s += 1
        if kmer in codon_count:
            codon_count[kmer] += 1
        else:
            codon_count[kmer] = 1
        i += 3

    for key, value in codon_count.items():
        codon_count[key] = value / s

    return codon_count

def read_fasta_file(file_path): #???
    sequences = []
    current_sequence = None
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if current_sequence:
                    sequences.append(current_sequence)
                sequence_id = line[1:]
                current_sequence = {'id': sequence_id, 'sequence': ''}
            else:
                current_sequence['sequence'] += line
        if current_sequence:
            sequences.append(current_sequence)
    return sequences

In [2]:
# File 1
file_path = 'Arabidopsis_thaliana_BHLH_gene_Family.fasta'
fasta = read_fasta_file(file_path)

ids = []
for id in fasta:
    ids.append(id['id'])

sequences = []
y = []
for seq in fasta:
    sequences.append(seq['sequence'])
    y.append(1)

# File 2
file_path = 'Arabidopsis_thaliana_CYP_gene_Family.fa'
fasta = read_fasta_file(file_path)

for seq in fasta:
    sequences.append(seq['sequence'])
    y.append(0)

for id in fasta:
    ids.append(id['id'])

In [8]:
np.count_nonzero(np.array(y) == 0)

218

In [47]:
kmer_result = []
for i in range(len(sequences)):
    kmer_result.append(codon(sequences[i], 1))
    #kmer_result.append(codon(sequences[i], 2))
    #kmer_result[i].update(codon(sequences[i], 1))  #???

In [50]:
v = DictVectorizer(sparse=False)
features = v.fit_transform(kmer_result)
feature_names = v.get_feature_names_out()
x = pd.DataFrame(features, columns=feature_names)
x.to_csv('kmer_features.csv', index=False)
x.rename(index=dict(zip(x.index, ids)), inplace=True)

In [51]:
x

,AAA,AAC,AAG,AAT,ACA,ACC,ACG,ACT,AGA,AGC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
AT1G51140.1 | Symbols: | basic helix-loop-helix (bHLH) DNA-binding superfamily protein | chr1:18943457-18945753 REVERSE LENGTH=2297,0.033987,0.014379,0.030065,0.026144,0.010458,0.005229,0.005229,0.014379,0.031373,0.007843,...,0.005229,0.020915,0.022222,0.009150,0.010458,0.015686,0.018301,0.018301,0.039216,0.041830
AT1G73830.1 | Symbols: BEE3 | BR enhanced expression 3 | chr1:27759975-27761447 FORWARD LENGTH=1473,0.042770,0.008147,0.010183,0.040733,0.010183,0.002037,0.004073,0.018330,0.034623,0.012220,...,0.004073,0.024440,0.018330,0.008147,0.006110,0.010183,0.036660,0.042770,0.018330,0.059063
"AT1G09530.1 | Symbols: PIF3, POC1, PAP3 | phytochrome interacting factor 3 | chr1:3076582-3079539 FORWARD LENGTH=2958",0.027383,0.013185,0.028398,0.020284,0.015213,0.007099,0.010142,0.020284,0.022312,0.013185,...,0.004057,0.033469,0.019270,0.017241,0.016227,0.022312,0.017241,0.027383,0.033469,0.041582
"AT1G49770.1 | Symbols: RGE1, ZOU | basic helix-loop-helix (bHLH) DNA-binding superfamily protein | chr1:18424578-18426782 FORWARD LENGTH=2205",0.038095,0.012245,0.021769,0.034014,0.016327,0.012245,0.010884,0.013605,0.017687,0.004082,...,0.005442,0.035374,0.014966,0.010884,0.016327,0.016327,0.031293,0.016327,0.023129,0.054422
AT1G68810.1 | Symbols: | basic helix-loop-helix (bHLH) DNA-binding superfamily protein | chr1:25861123-25863120 FORWARD LENGTH=1998,0.028529,0.016517,0.034535,0.013514,0.028529,0.012012,0.006006,0.012012,0.052553,0.021021,...,0.012012,0.019520,0.031532,0.006006,0.015015,0.016517,0.019520,0.021021,0.021021,0.049550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"AT5G14400.1 | Symbols: CYP724A1 | cytochrome P450, family 724, subfamily A, polypeptide 1 | chr5:4643521-4646382 FORWARD LENGTH=2862",0.049266,0.012579,0.017820,0.029350,0.017820,0.003145,0.005241,0.022013,0.027254,0.010482,...,0.001048,0.025157,0.022013,0.012579,0.015723,0.028302,0.023061,0.019916,0.020964,0.067086
"AT5G58860.1 | Symbols: CYP86A1, CYP86 | cytochrome P450, family 86, subfamily A, polypeptide 1 | chr5:23765814-23768049 REVERSE LENGTH=2236",0.030872,0.021477,0.009396,0.017450,0.009396,0.012081,0.018792,0.010738,0.021477,0.013423,...,0.009396,0.028188,0.029530,0.014765,0.014765,0.017450,0.013423,0.014765,0.021477,0.049664
"AT5G42650.1 | Symbols: AOS, CYP74A, DDE2 | allene oxide synthase | chr5:17097595-17099395 REVERSE LENGTH=1801",0.048333,0.015000,0.020000,0.025000,0.008333,0.015000,0.018333,0.011667,0.013333,0.008333,...,0.015000,0.013333,0.001667,0.006667,0.006667,0.015000,0.021667,0.036667,0.025000,0.031667
"AT5G05690.1 | Symbols: CPD, CYP90A, CYP90, CBB3, DWF3, CYP90A1 | Cytochrome P450 superfamily protein | chr5:1702688-1706780 REVERSE LENGTH=4093",0.045455,0.015396,0.023460,0.025660,0.012463,0.010997,0.009531,0.013930,0.019062,0.005865,...,0.005865,0.027126,0.021994,0.013196,0.010264,0.017595,0.024927,0.027126,0.024927,0.054985


In [53]:

label_encoder = LabelEncoder() # ?????

for col in x.columns:
    if x[col].dtype == 'object':
        x[col] = label_encoder.fit_transform(x[col])

y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_test = np.array(X_test)

In [54]:
knn = KNeighborsClassifier(n_neighbors=5)  

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

y_pred_labels = label_encoder.inverse_transform(y_pred)

print("Predicted labels:", y_pred_labels)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

precision = precision_score(y_test, y_pred)
print("Precision:", precision)

sensitivity = recall_score(y_test, y_pred)
print("Sensitivity (Recall):", sensitivity)

f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

Predicted labels: [0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0
 1 0]
Accuracy: 0.6710526315789473
Precision: 0.8333333333333334
Sensitivity (Recall): 0.40540540540540543
F1 Score: 0.5454545454545455


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
